In [1]:
# на всякий случай, ещё раз: в юпитере НЕ функционируют операторы exit() и quit(), так что не удивляйтесь, что после смерти игра продолжится

import random

# создаём искусственную задержку в игре: выполняет пустой цикл указанное количество раз
def busy_wait(delay_count):
    for _ in range(delay_count):
        pass

# Создаём визуальную шкалу отображения уровня сытости
def food_bar(food_value):
    if food_value >= 100:
        return "[██████████]"
    elif food_value >= 90:
        return "[█████████─]"
    elif food_value >= 80:
        return "[████████──]"
    elif food_value >= 70:
        return "[███████───]"
    elif food_value >= 60:
        return "[██████────]"
    elif food_value >= 50:
        return "[█████─────]"
    elif food_value >= 40:
        return "[████──────]"
    elif food_value >= 30:
        return "[███───────]"
    elif food_value >= 20:
        return "[██────────]"
    elif food_value >= 10:
        return "[█─────────]"
    else:
        return "[──────────]"

# Переменная для отслеживания текущего дня в игре
day = 0

# Функция форматирует инвентарь игрока для вывода на экран, преобразуя словарь в строки вида "[ресурс]: [количество]"
def resources(dictt):
    result = []
    for key, value in dictt.items():
        if key != "sekretik": # про секретик расскажет Дениска
            result.append(f"{key}: {value}")
    return "\n".join(result)

# Эту функцию включаем каждый раз, когда начинается новый день:
# 1) обновляются переменная дня и уровень еды
# 2) выводится инфа о состоянии инвентаря и доп. инфа в зависимости от дня
def day_change(day):
    day += 1
    player_state["food"] -= 10
    if player_state["food"] <= 0:
        print("\nКажется, вы забыли, что человеку нужно получать калории. Вы умерли от голода. Игра окончена.")
        exit()

     # Выводим информацию о новом дне
    print(f"\nДЕНЬ {day}\n————————————————————————————————————————————————————")
    print("Статус капсулы: " + player_state["capsule_status"]["status"])
    busy_wait(40000000)
    print("Требуют починки: " + ", ".join(player_state["capsule_status"]["requires_repairs"]))
    busy_wait(40000000)
    print(f"Состояние сытости/жажды: {player_state['food']}% {food_bar(player_state['food'])}")
    busy_wait(40000000)
    if player_state["inventory"]:
        print(f"У вас на руках:\n{resources(spisok_resursov)}")
    else:
        print("На данный момент в вашем инвентаре ничего нет.")
    busy_wait(40000000)
    
    if "oxygen_tank" in player_state and not player_state["oxygen_tank"] and day == 1:
        print("\nВНИМАНИЕ: У вас нет кислородного баллона для исследования глубоких мест!")
    busy_wait(40000000)

    if day == 10:
        print("Вы ощущаете признаки незнакомого заболевания, возможно, вы больны")
    if day == 15:
        print("Мутации, вызванные биосферой планеты приобрели необратимый характер, вас уже не спасти((")
        exit()
    return day

# Функция для крафта предметов
def craft_item(item):
    # Словарь с рецептами крафта: для каждого предмета указаны необходимые ресурсы
    # это работает так: у нас есть словарь crafts, а значения ключей этого словаря — ещё один словарь

    crafts = {
        "микросхема": {"кораллы": 2, "медь": 2},
        "кисл баллон высокой ёмкости": {"железо": 2},
        "нож": {"железо": 2},
        "острый раскалённый нож": {"нож": 1, "алмазы": 1, "микросхема": 1},
        "аптечки": {"синтетические волокна": 1},
        "синтетические волокна": {"водоросли": 2},
        "жареная рыба": {"водоросли": 1},
        "питательный коктейль": {"водоросли": 2},
        "консервы из морепродуктов": {"водоросли": 1, "нож": 0}
    }
    if item in crafts: # на всякий случай проверяем наличие предмета в словаре

        required = crafts[item] # если предмет найден, получаем его рецепт из словаря

        # первый случай: ресурсов не хватает
        for resource, amount in required.items(): # resource = ключи вложенного словаря, а amount = требуемое количество
            if spisok_resursov.get(resource, 0) < amount: # получаем количество ресурсов у игрока
                # метод .get() используется, чтобы вернуть 0, если одного или более ресуров нет в инвентаре
                print(f"Недостаточно {resource} для создания {item}.")
                return False # Возвращаем False, чтобы показать, что крафт невозможен

         # второй случай: ресурсов хватает
        for resource, amount in required.items():
            if resource == "нож" and item == "консервы из морепродуктов":
                pass # оператор pass гарантирует, что действие пройдёт, но нож останется (ибо нож не ломается при готовке консерв)
            else:
                spisok_resursov[resource] -= amount
                
        if item == "микросхема":
            spisok_resursov["микросхема"] = spisok_resursov.get("микросхема", 0) + 1
            print("Микросхема создана!")
        elif item == "кисл баллон высокой ёмкости":
            player_state["oxygen_tank"] = True
            print("Кислородный баллон создан! Теперь вы можете исследовать глубоководные места.")
        elif item == "нож":
            spisok_resursov["нож"] = spisok_resursov.get("нож", 0) + 1
            print("Нож создан! Он может пригодиться в будущем.")
        elif item == "острый раскалённый нож":
            spisok_resursov["острый раскалённый нож"] = spisok_resursov.get("острый раскалённый нож", 0) + 1
            print("Острый раскалённый нож создан!")
        elif item == "аптечки":
            if "изготовитель аптечек" in player_state["capsule_status"]["requires_repairs"]:
                print("вы не можете создать аптечку, пока изготовитель аптечек не функционирует")
                return False
            else:
                spisok_resursov["аптечки"] = spisok_resursov.get("аптечки", 0) + 1
                print("Аптечка создана! Она поможет при встрече с хищником.")
        elif item == "синтетические волокна":
            spisok_resursov["синтетические волокна"] = spisok_resursov.get("синтетические волокна", 0) + 1
            print("вы успешно переработали водоросли в синтетические волокна!")
        elif item == "жареная рыба":
            player_state["food"] += 20
            if player_state["food"] > 100:
                player_state["food"] = 100
            print("Вы приготовили жареную рыбу и восстановили 20% запаса еды!")
        elif item == "питательный коктейль":
            player_state["food"] += 30
            if player_state["food"] > 100:
                player_state["food"] = 100
            print("Вы приготовили питательный коктейль из водорослей и восстановили 30% запаса еды!")
        elif item == "консервы из морепродуктов":
            player_state["food"] += 25
            if player_state["food"] > 100:
                player_state["food"] = 100
            print("Вы приготовили консервы из морепродуктов и восстановили 25% запаса еды!")
        return True
    else:
        print("Такого предмета нельзя создать.")
        return False

player_state = {
    "capsule_status": {
        "status": "сломана.",
        "requires_repairs": ["радио", "радар(необходим для поиска редких ресурсов)", "изготовитель аптечек"]
    },
    "food": 90,
    "inventory": {},
    "oxygen_tank": False, # проверяем, есть ли кислородный баллон
    "has_visited_cave": False # проверяем, заплывал ли уже Олег в пещеру
}

spisok_resursov = { # непосредственно список ресурсов Олега
    "медь": 0,
    "железо": 0,
    "водоросли": 0,
    "алмазы": 0,
    "кораллы": 0,
    "магнетит": 0,
    "редкие ресурсы": 0,
    "микросхема": 0,
    "синтетические волокна": 0,
    "нож": 0,
    "острый раскалённый нож": 0,
    "аптечки": 0,
    "sekretik": ";)"
}

player_state["inventory"] = spisok_resursov

item_kill = 0 # готовим сюрприз игроку

print("ТРЕВОГА. ПОКИНУТЬ КОРАБЛЬ.")
busy_wait(70000000)
print("\n...")
busy_wait(70000000)
print("\n...")
busy_wait(70000000)
print("\n...")
busy_wait(70000000)
print("\n*Вы погрузились в спасательную капсулу и успешно десантировались на планету*")
busy_wait(70000000)
print("\n\nКорабль потерпел крушение из-за внезапной космической аномалии, которая повергла его в хаос.")
busy_wait(70000000)
print("\nВспышки света, крики и нескончаемый шум разрушения ознаменовали конец вашего судна.")
busy_wait(70000000)
print("\nВы — Ибрагим, отважный исследователь и пилот, чей разум всё ещё борется с шоком от произошедшего.")
busy_wait(70000000)
print("\nВ капсуле, где вы сейчас находитесь, пространство тесно и темно, а каждый звук отзывается эхом по холодным металлическим стенам.")
busy_wait(70000000)
print("\nВы чувствуете смешанные эмоции: страх, решимость и странное спокойствие, осознавая, что вы ещё живы.")
busy_wait(70000000)
print("\nЭто начало вашего пути к выживанию и поиску спасения.")
busy_wait(70000000)
print("Добро пожаловать в...")
busy_wait(50000000)
print("\n███████╗██╗  ██╗ ██████╗ ███╗   ███╗ █████╗ ██████╗  ██████╗  ██████╗ ███╗   ██╗███████╗██████╗")
busy_wait(40000000)
print("██╔════╝╚██╗██╔╝██╔═══██╗████╗ ████║██╔══██╗██╔══██╗██╔═══██╗██╔═══██╗████╗  ██║██╔════╝██╔══██╗")
busy_wait(40000000)
print("█████╗   ╚███╔╝ ██║   ██║██╔████╔██║███████║██████╔╝██║   ██║██║   ██║██╔██╗ ██║█████╗  ██║  ██║")
busy_wait(40000000)
print("██╔══╝   ██╔██╗ ██║   ██║██║╚██╔╝██║██╔══██║██╔══██╗██║   ██║██║   ██║██║╚██╗██║██╔══╝  ██║  ██║")
busy_wait(40000000)
print("███████╗██╔╝ ██╗╚██████╔╝██║ ╚═╝ ██║██║  ██║██║  ██║╚██████╔╝╚██████╔╝██║ ╚████║███████╗██████╔╝")
busy_wait(40000000)
print("╚══════╝╚═╝  ╚═╝ ╚═════╝ ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝  ╚═╝ ╚═════╝  ╚═════╝ ╚═╝  ╚═══╝╚══════╝╚═════╝")
busy_wait(50000000)
print("Вы оказались на неизвестной планете после катастрофы космического корабля.", end="", flush=True)
busy_wait(100000000)
print("\nВаша задача — выжить в условиях ограниченных ресурсов и исследовать окружающую среду, чтобы найти путь к спасению.", end="", flush=True)
busy_wait(100000000)
print("\nКаждый день вам предстоит выбирать одно действие из предложенных вариантов.", end="", flush=True)
busy_wait(100000000)
print("\nПри этом с каждым днем уровень сытости и жажды будет падать на 10%, а ваше состояние будет зависеть от сделанных выборов.", end="", flush=True)
busy_wait(100000000)
print("\nУправляйте своим инвентарем, исследуйте территорию и восстанавливайте свою капсулу, чтобы увеличить шансы на выживание и спасение.", end="", flush=True)
busy_wait(100000000)

day = day_change(day)
busy_wait(100000000)
print("С каждым действием, которое вы выбираете, вы повышаете шанс своего выживания. Что конкретно делать — остаётся за вами.")
busy_wait(40000000)
print("————————————————————————————————————————————————————")
busy_wait(100000000)
print("Вы можете попытаться в первую очередь выйти наружу или сосредоточиться на анализе состояния своей спасательной капсулы.")
busy_wait(40000000)
print("1. Выйти наружу.")
busy_wait(40000000)
print("2. Исследовать спасательную капсулу.")
busy_wait(100000000)

while True:
    choice = input("Введите номер действия: ").strip()
    while choice not in {"1", "2"}:
        choice = input("Неверный выбор. Введите 1 или 2: ").strip()

    if choice == "1": # идём наружу
        print("\nВы решаете исследовать местность.")
        busy_wait(40000000)
        print("Осторожно выходя из капсулы, вы ощущаете плотность воды, окружающей вас. Вокруг разрастаются странные морские растения, а вдалеке виднеются силуэты неизвестных существ.")
        busy_wait(40000000)
        print("На видимом горизонте вы замечаете, что рядом с капсулой вода относительно мелкая. Это как никогда полезно, ведь таким образом у вас получается заполучить немалое количество ресурсов.")
        busy_wait(40000000)
        
        if random.random() < 0.2:  # с шансом 1 к 5 нас может атаковать хищник
            print("\nВнезапно на вас нападает подводный хищник!")
            if spisok_resursov["аптечки"] > 0:
                spisok_resursov["аптечки"] -= 1
                print("Вы использовали аптечку и избежали урона.")
            else:
                player_state["food"] -= 20
                print("У вас не было аптечек, и хищник нанёс вам урон. В борьбе с ним вы потеряли много энергии. Вы потеряли 20% еды.")
                if player_state["food"] <= 0:
                    print("В борьбе с хищником вы потеряли много энергии. Вы умерли от недостатка сил после атаки. Игра окончена.")
                    exit()

        # второй выбор: что делать после выхода наружу
        print("Прямо сейчас вы можете сосредоточиться на исследовании местности: растений, рыб, доступных ископаемых. Либо сосредоточиться исключительно на сборе ресурсов.")
        busy_wait(40000000)
        print("1. Изучать местность.\n2. Собирать ресурсы")
            
        while True: # Выбор: собирать ресы или исследовать местность
            inner_choice = input("Введите номер действия: ").strip()
            if inner_choice == "2":
                print("\nВы решаете собирать ресурсы.") # если решили собирать ресы
                busy_wait(40000000)
                print("Вокруг, на дне и на склонах подземных гор можно увидеть различные камни. А чуть дальше зеленеют высокие, длинные водоросли.")
                busy_wait(40000000)
                print("Если проплыть чуть дальше, можно собрать кораллы, которые могут пригодиться в починке основных модулей капсулы.")
                busy_wait(40000000)
                print("Чем вы займетесь в первую очередь: добычей камней (если очистить их от примесей, вы сможете получить полезные материалы); собиранием кораллов или поплывёте за водорослями?")
                busy_wait(40000000)
                print("1. Собирать камни.\n2. Нарезать кораллы.\n3. Поплыть за водорослями.")
                while True:
                    second_inner_choice = input("Введите номер действия: ").strip()
                    if second_inner_choice == "1":
                        print("Вам удается найти несколько камней, содержащих железо и медь.")
                        spisok_resursov["медь"] += 2
                        spisok_resursov["железо"] += 2
                        busy_wait(40000000)
                        print("У вас еще есть возможность собрать водоросли или кораллы.")
                        busy_wait(40000000)
                        print("1. Нарезать кораллы.\n2. Поплыть за водорослями.")
                        while True:
                            third_inner_choice = input("Введите номер действия: ").strip()
                            if third_inner_choice == "1":
                                print("Вы успешно срезаете несколько пластин коралла.")
                                spisok_resursov["кораллы"] += 2
                                busy_wait(40000000)
                                break
                            elif third_inner_choice == "2":
                                print("Вы успешно срезаете несколько стеблей водорослей.")
                                spisok_resursov["водоросли"] += 2
                                busy_wait(40000000)
                                break
                            else:
                                print('Ошибка ввода. Введите "1" или "2".')
                    elif second_inner_choice == "2":
                        print("Вы успешно срезаете несколько пластин коралла.")
                        spisok_resursov["кораллы"] += 2
                        busy_wait(40000000)
                        print("У вас еще есть возможность собрать водоросли или камни.")
                        busy_wait(40000000)
                        print("1. Собирать камни.\n2. Поплыть за водорослями.")
                        while True:
                            fourth_inner_choice = input("Введите номер действия: ").strip()
                            if fourth_inner_choice == "1":
                                print("Вам удается найти несколько камней, содержащих железо и медь.")
                                spisok_resursov["медь"] += 2
                                spisok_resursov["железо"] += 2
                                busy_wait(40000000)
                                break
                            elif fourth_inner_choice == "2":
                                print("Вы успешно срезаете несколько стеблей водорослей.")
                                spisok_resursov["водоросли"] += 2
                                busy_wait(40000000)
                                break
                            else:
                                print('Ошибка ввода. Введите "1" или "2".')
                    elif second_inner_choice == "3":
                        print("Вы успешно срезаете несколько стеблей водорослей.")
                        spisok_resursov["водоросли"] += 2
                        busy_wait(40000000)
                        print("У вас еще есть возможность собрать камни или кораллы.")
                        busy_wait(40000000)
                        print("1. Нарезать кораллы.\n2. Собирать камни.")
                        while True:
                            fifth_inner_choice = input("Введите номер действия: ").strip()
                            if fifth_inner_choice == "1":
                                print("Вы успешно срезаете несколько пластин коралла.")
                                spisok_resursov["кораллы"] += 2
                                busy_wait(40000000)
                                break
                            elif fifth_inner_choice == "2":
                                print("Вам удается найти несколько камней, содержащих железо и медь.")
                                spisok_resursov["медь"] += 2
                                spisok_resursov["железо"] += 2
                                busy_wait(40000000)
                                break
                            else:
                                print('Ошибка ввода. Введите "1" или "2".')
                    else:
                        print('Ошибка ввода. Введите "1", "2" или "3".')
                        continue
                    break
                print("С помощью небольших скоплений водорослей вам удаётся создать импровизированную сеть. Благодаря ней вы ловите несколько рыб.")
                busy_wait(40000000)
                
                food_found = random.randint(10, 25) # если собираем водоросли, то заодно ловим рыбок и получаем от 10 до 25% еды
                player_state["food"] += food_found
                if player_state["food"] > 100:
                    player_state["food"] = 100
                print(f"Вам удалось поймать рыбу и восстановить {food_found}% запаса еды!")
                busy_wait(40000000)
                
                print("Стало темнеть. Пора возвращаться в капсулу.")
                busy_wait(100000000)
                break
            elif inner_choice == "1": # если выбрали исследовать местность
                print("Вы решаете изучить местность вокруг.")
                busy_wait(40000000)
                print("Вы замечаете, что некоторые растения испускают слабое свечение, а мелкие рыбки не боятся вас.")
                busy_wait(40000000)
                print("Вдали вы замечаете нечто интересное — возможно, вход в подводную пещеру.")
                busy_wait(40000000)
                print("1. Исследовать\n2. Вернуться в капсулу")
                investigate_choice = input("Введите номер действия: ").strip()
                if investigate_choice == "1":
                    print("Вы подплываете ближе.")
                    busy_wait(40000000)
                    if player_state["oxygen_tank"]:
                        print("С кислородным баллоном вы погружаетесь и находите пещеру.")
                        busy_wait(40000000)
                        
                        if random.random() < 0.5:
                            food_found = random.randint(15, 30)
                            player_state["food"] += food_found
                            if player_state["food"] > 100:
                                player_state["food"] = 100
                            print(f"В пещере вы обнаружили редких морских обитателей и смогли добыть пищу, восстановив {food_found}% запаса еды!")
                            busy_wait(40000000)
                    else:
                        print("Без кислородного баллона вы не можете дышать под водой.")
                        busy_wait(40000000)
                        print("Вы теряете сознание и умираете. Игра окончена.")
                        exit()
                elif investigate_choice == "2":
                    print("Вы возвращаетесь в капсулу.")
                    busy_wait(40000000)
                print("День подошёл к концу.")
                busy_wait(100000000)
                break
            else:
                print('Ошибка ввода. Введите "1" или "2".')
        break
                    # здесь заканчивается цикл если выбираем выйти наружу
    elif choice == "2":
        print("\nВы начинаете исследовать капсулу.")
        busy_wait(40000000)
        print("Повреждены: радио, радар, изготовитель аптечек.")
        busy_wait(40000000)
        print("В попытках починить капсулу вы провели весь день, зачем?") # троллинг игрока
        busy_wait(40000000)
        print("у вас нет ресурсов для починки капсулы, попробуйте найти их вне её")
        break
        
while True:
    day = day_change(day)
    if item_kill == 4: # сюрприз для игрока
        busy_wait(40000000)
        print("вы просыпаетесь от сильной тряски, кажется, будто само небо разверглось.")
        busy_wait(40000000)
        print("похоже тряска изходит откуда-то из глубин океана.")
        busy_wait(40000000)
        print("вам становится непосебе, однако любопытство побеждает и вы решаете взглянуть на источник шума.")
        busy_wait(40000000)
        print("выйдя из капсулы отплыв немного вы замечаете огромную пропасть, которой раньше не было.")
        busy_wait(40000000)
        print("пропасть росла и вы захотели одним глазком посмотреть внутрь.")
        busy_wait(40000000)
        print("оттуда что-то поднималось...")
        busy_wait(40000000)
        print("через несколько мгновений перед вами явилось огромное неописуемое существо...")
        busy_wait(40000000)
        print("1. что...")
        print("2. С!«W2Єџз,hBU*«ЂV*Vія‚&]ИVќs‡—jія‚&Q;PxН•»TН")
        while True:
            chto = input()
            if chto == 1:
                print("。。。")
                busy_wait(40000000)
                print("。。。")
                busy_wait(40000000)
                print("。。。")
                busy_wait(40000000)
                print("。。。")
                busy_wait(40000000)
                print("。。。")
                busy_wait(40000000)
                print("4–{®†Ґв№†Ў=°Т$ГЦ,ЂС¬лHЙ…Юse0л14'ПC\Е‡lмАХЎ*ХФ ЧXХъVГ%FџЉР¶jQ d™m: -v`i›џCP ЁЖl1ма—8ЂЦsщf¬ЇЧЏ#ызcj$AOЖ PЎ†ж")
                busy_wait(40000000)
                print("Эм... похоже вы сошли с ума?")
                exit()
            elif chto == 2:
                print("Кажется, вы сошли с ума?")
                exit() # увы
            else:
                print("К сожалению, необходимо выбрать какой - то из двух вариантов, в такой ситуации, ничего другое вы сказать физически не можете")
    
    # через эту переменную отслеживаем, был ли совершён крафт в текущий день
    crafting_used = False
    
    while True:
        if not crafting_used:
            print("\nВы можете совершить одно действие за день:")
        else:
            print("\nВы уже использовали крафт. Выберите основное действие на день:")
        
        busy_wait(40000000)
        print("1. Исследовать окрестности капсулы")
        busy_wait(40000000)
        print("2. Попытаться починить капсулу")
        busy_wait(40000000)

        # мы можем крафтить единожды в день
        if not crafting_used:
            print("3. Создать предмет (не тратит день)")
            busy_wait(40000000)

        # при починке радара можем его запустить
        if "радар" not in player_state["capsule_status"]["requires_repairs"] and player_state["capsule_status"]["status"] == "отремонтирована.":
            if not crafting_used:
                print("4. Запустить радар")
            else:
                print("3. Запустить радар")
            busy_wait(40000000)

        # при починке всей капсулы мы можем отправить сигнал бедствия, что ведёт к хорошей концовке
        if player_state["capsule_status"]["status"] == "отремонтирована." and "радио" not in player_state["capsule_status"]["requires_repairs"]:
            if not crafting_used:
                print("5. Отправить сигнал бедствия")
            else:
                print("4. Отправить сигнал бедствия")
            busy_wait(40000000)
        
        extra_choice = input("Введите номер действия: ").strip()

        # исследовать местность (в т. ч. пещеру) и собирать ресы можно в любой день
        if extra_choice == "1":
            print("\nВы решаете исследовать местность.")
            busy_wait(40000000)
            if random.random() < 0.2:
                print("\nВнезапно на вас нападает подводный хищник!")
                if spisok_resursov["аптечки"] > 0:
                    spisok_resursov["аптечки"] -= 1
                    print("Вы использовали аптечку и избежали урона.")
                else:
                    player_state["food"] -= 20
                    print("У вас не было аптечек, и хищник нанёс вам урон. В борьбе с ним вы потеряли много энергии. Вы потеряли 20% еды.")
                    if player_state["food"] <= 0:
                        print("В борьбе с хищником вы потеряли много энергии. Вы умерли от недостатка сил после атаки. Игра окончена.")
                        exit()
            print("Прямо сейчас вы можете сосредоточиться на исследовании местности: растений, рыб, доступных ископаемых. Либо сосредоточиться исключительно на сборе ресурсов.")
            busy_wait(40000000)
            print("1. Изучать местность.\n2. Собирать ресурсы")
       
            while True:
                true_extra_choice = input("Введите номер действия: ").strip()
                if true_extra_choice == "2":
                    print("\nВы решаете собирать ресурсы.")
                    busy_wait(40000000)
                    print("Вокруг, на дне и на склонах подводных гор можно увидеть различные камни. А чуть дальше зеленеют высокие, длинные водоросли.")
                    busy_wait(40000000)
                    print("Если проплыть чуть дальше, можно собрать кораллы, которые могут пригодиться в починке основных модулей капсулы.")
                    busy_wait(40000000)
                    print("Чем вы займетесь в первую очередь: добычей камней (если очистить их от примесей, вы сможете получить полезные материалы); собиранием кораллов или поплывёте за водорослями?")
                    busy_wait(40000000)
                    print("1. Собирать камни.\n2. Нарезать кораллы.\n3. Поплыть за водорослями.")
                    while True:
                        second_true_extra_choice = input("Введите номер действия: ").strip()
                        if second_true_extra_choice == "1":
                            print("Вам удается найти несколько камней, содержащих железо и медь.")
                            spisok_resursov["медь"] += 2
                            spisok_resursov["железо"] += 2
                            busy_wait(40000000)
                            print("У вас еще есть возможность собрать водоросли или кораллы.")
                            busy_wait(40000000)
                            print("1. Нарезать кораллы.\n2. Поплыть за водорослями.")
                            while True:
                                third_true_extra_choice = input("Введите номер действия: ").strip()
                                if third_true_extra_choice == "1":
                                    print("Вы успешно срезаете несколько пластин коралла.")
                                    spisok_resursov["кораллы"] += 2
                                    busy_wait(40000000)
                                    break
                                elif third_true_extra_choice == "2":
                                    print("Вы успешно срезаете несколько стеблей водорослей.")
                                    spisok_resursov["водоросли"] += 2
                                    busy_wait(40000000)
                                    break
                                else:
                                    print('Ошибка ввода. Введите "1" или "2".')
                        elif second_true_extra_choice == "2":
                            print("Вы успешно срезаете несколько пластин коралла.")
                            spisok_resursov["кораллы"] += 2
                            busy_wait(40000000)
                            print("У вас еще есть возможность собрать водоросли или камни.")
                            busy_wait(40000000)
                            print("1. Собирать камни.\n2. Поплыть за водорослями.")
                            while True:
                                fourth_true_extra_choice = input("Введите номер действия: ").strip()
                                if fourth_true_extra_choice == "1":
                                    print("Вам удается найти несколько камней, содержащих железо и медь.")
                                    spisok_resursov["медь"] += 2
                                    spisok_resursov["железо"] += 2
                                    busy_wait(40000000)
                                    break
                                elif fourth_true_extra_choice == "2":
                                    print("Вы успешно срезаете несколько стеблей водорослей.")
                                    spisok_resursov["водоросли"] += 2
                                    busy_wait(40000000)
                                    break
                                else:
                                    print('Ошибка ввода. Введите "1" или "2".')
                        elif second_true_extra_choice == "3":
                            print("Вы успешно срезаете несколько стеблей водорослей.")
                            spisok_resursov["водоросли"] += 2
                            busy_wait(40000000)
                            print("У вас еще есть возможность собрать камни или кораллы.")
                            busy_wait(40000000)
                            print("1. Нарезать кораллы.\n2. Собирать камни.")
                            while True:
                                fifth_true_extra_choice = input("Введите номер действия: ").strip()
                                if fifth_true_extra_choice == "1":
                                    print("Вы успешно срезаете несколько пластин коралла.")
                                    spisok_resursov["кораллы"] += 2
                                    busy_wait(40000000)
                                    break
                                elif fifth_true_extra_choice == "2":
                                    print("Вам удается найти несколько камней, содержащих железо и медь.")
                                    spisok_resursov["медь"] += 2
                                    spisok_resursov["железо"] += 2
                                    busy_wait(40000000)
                                    break
                                else:
                                    print('Ошибка ввода. Введите "1" или "2".')
                        else:
                            print('Ошибка ввода. Введите "1", "2" или "3".')
                        break
                    print("С помощью небольших скоплений водорослей вам удаётся создать импровизированную сеть. Благодаря ней вы ловите несколько рыб.")
                    busy_wait(40000000)
                    
                    food_found = random.randint(10, 25)
                    player_state["food"] += food_found
                    if player_state["food"] > 100:
                        player_state["food"] = 100
                    print(f"Вам удалось поймать рыбу и восстановить {food_found}% запаса еды!")
                    busy_wait(40000000)
                    
                    print("Стало темнеть. Пора возвращаться в капсулу.")
                    busy_wait(100000000)
                    break
                
                elif true_extra_choice == "1":
                    print("Вы решаете изучить местность вокруг.")
                    busy_wait(40000000)
                    print("Вы замечаете, что некоторые растения испускают слабое свечение, а мелкие рыбки не боятся вас.")
                    busy_wait(40000000)
                    print("Вдали вы замечаете нечто интересное — возможно, вход в подводную пещеру.")
                    busy_wait(40000000)
                    print("1. Исследовать\n2. Вернуться в капсулу")
                    new_investigate_choice = input("Введите номер действия: ").strip()
                    if new_investigate_choice == "1":
                        print("Вы подплываете ближе.")
                        busy_wait(40000000)
                        if player_state["oxygen_tank"]:
                            print("С кислородным баллоном вы погружаетесь и находите пещеру с ресурсами.")
                            busy_wait(40000000)
                            print("В глубине пещеры обитает крупный хищник.")
                            busy_wait(40000000)
                            print("у вас всё ещё есть возможность вернуться.")
                            busy_wait(40000000)
                            while True:
                                vybor = input("1. Вернуться домой(скорее всего перед встречей с хищником может понадобиться обзавестись оружием)\n2. Рискнуть" ).strip()
                                if vybor == "1":
                                    print("Вы возвращаетесь в капсулу.")
                                    busy_wait(40000000)
                                    break
                                elif vybor == "2":
                                    print("Вы спускаетесь в пещеру.")
                                    busy_wait(40000000)
                                    chance = random.random()
                                    if spisok_resursov["нож"] >= 1:
                                        if chance < 0.6:
                                            print("\nВнезапно на вас нападает подводный хищник!")
                                            if spisok_resursov["аптечки"] > 0:
                                                glupyi_vybor = input("использовать аптечку? (1. да; 2. нет.)").strip()
                                                if glupyi_vybor == "1":
                                                    spisok_resursov["аптечки"] -= 1
                                                    print("Вы использовали аптечку и избежали урона.")
                                                elif glupyi_vybor == "2":
                                                    spisok_resursov["аптечки"] -= 1
                                                    print("Глупый выбор, вы всё равно использовали аптечку, так как инстинкт самосохранения оказался сильнее")
                                                else:
                                                    print('Ошибка ввода. Введите "1" или "2".')
                                            else:
                                                print("Вы умерли после атаки. Игра окончена.")
                                                exit()
                                        elif 0.6 <= chance <= 0.8:
                                            print("Вам удаётся отпугнуть хищника, но вы всё равно не можете добраться до скопления ресурсов на дне пещеры.")
                                        elif chance > 0.8:
                                            print("Вам удаётся убить хищника и собрать немного ресурсов со дна пещеры.")
                                            spisok_resursov["магнетит"] += 1
                                            spisok_resursov["алмазы"] += 1
                                            
                                            food_found = random.randint(20, 35)
                                            player_state["food"] += food_found
                                            if player_state["food"] > 100:
                                                player_state["food"] = 100
                                            print(f"Вы разделали тушу хищника и смогли добыть пищи, восстановив {food_found}% запаса еды!")
                                            
                                            item_kill += 1
                                            busy_wait(40000000)
                                    elif spisok_resursov["острый раскалённый нож"] >= 1:
                                        if chance < 0.3:
                                            print("\nВнезапно на вас нападает подводный хищник!")
                                            if spisok_resursov["аптечки"] > 0:
                                                glupyi_vybor = input("использовать аптечку? (1. да; 2. нет.)").strip()
                                                if glupyi_vybor == "1":
                                                    spisok_resursov["аптечки"] -= 1
                                                    print("Вы использовали аптечку и избежали урона.")
                                                elif glupyi_vybor == "2":
                                                    spisok_resursov["аптечки"] -= 1
                                                    print("Глупый выбор, вы всё равно использовали аптечку, так как инстинкт самосохранения оказался сильнее")
                                                else:
                                                    print('Ошибка ввода. Введите "1" или "2".')
                                            else:
                                                print("Вы умерли после атаки. Игра окончена.")
                                                exit()
                                        
                                        elif 0.3 <= chance <= 0.5:
                                            print("Вам удаётся отпугнуть хищника, но вы всё равно не можете добраться до скопления ресурсов на дне пещеры.")
                                        elif chance > 0.5:
                                            print("Вам удаётся убить хищника и собрать немного ресурсов со дна пещеры.")
                                            spisok_resursov["магнетит"] += 1
                                            spisok_resursov["алмазы"] += 1
                                            
                                            food_found = random.randint(20, 35)
                                            player_state["food"] += food_found
                                            if player_state["food"] > 100:
                                                player_state["food"] = 100
                                            print(f"Вы разделали тушу хищника и смогли добыть пищи, восстановив {food_found}% запаса еды!")
                                            
                                            item_kill += 1
                                            busy_wait(40000000)
                                    else:
                                        if chance < 0.8:
                                            print("\nВнезапно на вас нападает подводный хищник!")
                                            if spisok_resursov["аптечки"] > 0:
                                                glupyi_vybor = input("использовать аптечку? (1. да; 2. нет.)").strip()
                                                if glupyi_vybor == "1":
                                                    spisok_resursov["аптечки"] -= 1
                                                    print("Вы использовали аптечку и избежали урона.")
                                                elif glupyi_vybor == "2":
                                                    spisok_resursov["аптечки"] -= 1
                                                    print("Глупый выбор, вы всё равно использовали аптечку, так как инстинкт самосохранения оказался сильнее")
                                                else:
                                                    print('Ошибка ввода. Введите "1" или "2".')
                                            else:
                                                print("Вы умерли после атаки. Игра окончена.")
                                                exit()
                                        elif 0.8 <= chance:
                                            print("Вам удается увернуться от атаки и ретировться из пещеры")
                                break
                            player_state["has_visited_cave"] = True
                            busy_wait(40000000)
                        else:
                            print("Без кислородного баллона вы не можете дышать под водой.")
                            busy_wait(40000000)
                            print("Вы теряете сознание и умираете. Игра окончена.")
                            exit()
                    elif new_investigate_choice == "2":
                        print("Вы возвращаетесь в капсулу.")
                        busy_wait(40000000)
                    print("День подошёл к концу.")
                    busy_wait(100000000)
                    break
                else:
                    print('Ошибка ввода. Введите "1" или "2".')
            break

        # на второй день появляется возможность чинить системы капсулы
        elif extra_choice == "2":
            if player_state["capsule_status"]["requires_repairs"]:
                print("Какую систему чинить?")
                print("1. Радио (1 микросхема, 2 медь, редкие ресурсы)")
                print("2. Радар (1 микросхема, 2 железо, 1 магнетит)")
                print("3. Изготовитель аптечек (1 микросхема, 1 синтетические волокна)")
                print("4. починить капсулу (2 медь, 2 железо, 1 микросхема, 1 алмаз, 1 магнетит, редкие ресурсы)")
                repair_choice = input("Введите номер: ").strip()
                if repair_choice == "1" and "радио" in player_state["capsule_status"]["requires_repairs"] and player_state["capsule_status"]["status"] == "отремонтирована.":
                    if spisok_resursov["микросхема"] >= 1 and spisok_resursov["медь"] >= 2 and spisok_resursov["редкие ресурсы"] >= 1:
                        spisok_resursov["микросхема"] -= 1
                        spisok_resursov["медь"] -= 2
                        spisok_resursov["редкие ресурсы"] -= 1
                        player_state["capsule_status"]["requires_repairs"].remove("радио")
                        print("Радио отремонтировано!")
                        busy_wait(40000000) 
                    else:
                        print("Недостаточно ресурсов. В попытках хоть что-то исправить или починить вы провели весь день, вам стоит лучше рассчитывать время.")
                elif repair_choice == "1" and "радио" not in player_state["capsule_status"]["requires_repairs"]:
                    print("Радио функционирует!")
                    
                elif repair_choice == "2" and "радар" in player_state["capsule_status"]["requires_repairs"]:
                    if spisok_resursov["микросхема"] >= 1 and spisok_resursov["железо"] >= 2 and spisok_resursov["магнетит"] >= 1:
                        spisok_resursov["микросхема"] -= 1
                        spisok_resursov["железо"] -= 2
                        spisok_resursov["магнетит"] -= 1
                        player_state["capsule_status"]["requires_repairs"].remove("радар")
                        print("Радар отремонтирован!")
                        busy_wait(40000000)
                    else:
                        print("Недостаточно ресурсов. В попытках хоть что-то исправить или починить вы провели весь день, вам стоит лучше рассчитывать время.")
                elif repair_choice == "2" and "радар" not in player_state["capsule_status"]["requires_repairs"]:
                    print("Радар функционирует!")
                    
                elif repair_choice == "3" and "изготовитель аптечек" in player_state["capsule_status"]["requires_repairs"]:
                    if spisok_resursov["микросхема"] >= 1 and spisok_resursov["синтетические волокна"] >= 1:
                        spisok_resursov["микросхема"] -= 1
                        spisok_resursov["синтетические волокна"] -= 1
                        player_state["capsule_status"]["requires_repairs"].remove("изготовитель аптечек")
                        print("Изготовитель аптечек отремонтирован!")
                        busy_wait(40000000)
                    else:
                        print("Недостаточно ресурсов. В попытках хоть что-то исправить или починить вы провели весь день, вам стоит лучше рассчитывать время.")
                elif repair_choice == "3" and "изготовитель аптечек" not in player_state["capsule_status"]["requires_repairs"]:
                    print("Изготовитель аптечек функционирует!")
                    
                elif repair_choice == "4" and player_state["capsule_status"]["status"] == "сломана.":
                    if spisok_resursov["микросхема"] >= 1 and spisok_resursov["медь"] >= 2 and spisok_resursov["редкие ресурсы"] >= 1 and spisok_resursov["железо"] >= 2 and spisok_resursov["магнетит"] >= 1 and spisok_resursov["алмазы"] >= 1:
                        spisok_resursov["микросхема"] -= 1
                        spisok_resursov["медь"] -= 2
                        spisok_resursov["редкие ресурсы"] -= 1
                        spisok_resursov["железо"] -= 2
                        spisok_resursov["магнетит"] -= 1
                        spisok_resursov["алмазы"] -= 1
                        player_state["capsule_status"]["status"] = "отремонтирована."
                        print("Капсула отремонтирована!")
                        busy_wait(40000000)
                    else:
                        print("Недостаточно ресурсов. В попытках хоть что-то исправить или починить вы провели весь день, вам стоит лучше рассчитывать время.")
                elif repair_choice == "4" and player_state["capsule_status"]["status"] != "сломана.":
                    print("Капсула функционирует!")
            break

        # если выбрали создать предмет:
        elif extra_choice == "3" and not crafting_used:
            print("Что создать?")
            print("1. Микросхема (2 коралла, 2 меди)")
            print("2. Кислородный баллон (2 железа)")
            print("3. Нож (2 железа)")
            print("4. Острый раскалённый нож (1 нож, 1 алмаз, 1 микросхема)")
            print("5. Аптечки (1 синтетическое волокно)")
            print("6. Переработать водоросли в волокна (2 водоросли)")
            print("7. Жареная рыба (1 водоросль) - восстановит 20% еды")
            print("8. Питательный коктейль (2 водоросли) - восстановит 30% еды")
            print("9. Консервы из морепродуктов (1 водоросль, требуется нож) - восстановит 25% еды")
            print("0. Вернуться назад.")
            
            craft_choice = input("Введите номер (0 - для возврата): ").strip()
            if craft_choice == "0":
                print("Возврат в основное меню.")
                continue
            elif craft_choice == "1":
                if craft_item("микросхема"):
                    crafting_used = True
            elif craft_choice == "2":
                if craft_item("кисл баллон высокой ёмкости"):
                    crafting_used = True
            elif craft_choice == "3":
                if craft_item("нож"):
                    crafting_used = True
            elif craft_choice == "4":
                if craft_item("острый раскалённый нож"):
                    crafting_used = True
            elif craft_choice == "5":
                if craft_item("аптечки"):
                    crafting_used = True
            elif craft_choice == "6":
                if craft_item("синтетические волокна"):
                    crafting_used = True
            elif craft_choice == "7":
                if craft_item("жареная рыба"):
                    crafting_used = True
            elif craft_choice == "8":
                if craft_item("питательный коктейль"):
                    crafting_used = True
            elif craft_choice == "9":
                if spisok_resursov.get("нож", 0) > 0:
                    if craft_item("консервы из морепродуктов"):
                        crafting_used = True
                else:
                    print("Вам нужен нож для приготовления консервов!")
            else:
                print("Неверный выбор.")

        # если выбрали запустить радар:
        elif (extra_choice == "4" and not crafting_used) or (extra_choice == "3" and crafting_used):
            if "радар" not in player_state["capsule_status"]["requires_repairs"] and player_state["capsule_status"]["status"] == "отремонтирована.":
                print("Вы активируете радар...")
                busy_wait(40000000)  

                # с шансом 1 к 2 радар может обнаружить редкие ресурсы, необходимые для финальной починки капсулы
                if random.random() < 0.5:
                    print("Радар обнаружил скопление редких ресурсов неподалёку!")
                    spisok_resursov["редкие ресурсы"] += 1 
                else:
                    print("Радар не обнаружил ничего интересного.")
                break
            else:
                print("Неверный выбор.")

        # если выбрали отправить сигнал бедствия:
        elif (extra_choice == "5" and not crafting_used) or (extra_choice == "4" and crafting_used):
            if player_state["capsule_status"]["status"] == "отремонтирована." and "радио" not in player_state["capsule_status"]["requires_repairs"]:
                print("\nСигнал бедствия отправлен. Вы выжили! Игра окончена.")
                exit()
            else:
                print("Неверный выбор.")
        
        else:
            print("Неверный выбор.")

ТРЕВОГА. ПОКИНУТЬ КОРАБЛЬ.

...

...

...

*Вы погрузились в спасательную капсулу и успешно десантировались на планету*


Корабль потерпел крушение из-за внезапной космической аномалии, которая повергла его в хаос.

Вспышки света, крики и нескончаемый шум разрушения ознаменовали конец вашего судна.

Вы — Ибрагим, отважный исследователь и пилот, чей разум всё ещё борется с шоком от произошедшего.

В капсуле, где вы сейчас находитесь, пространство тесно и темно, а каждый звук отзывается эхом по холодным металлическим стенам.

Вы чувствуете смешанные эмоции: страх, решимость и странное спокойствие, осознавая, что вы ещё живы.

Это начало вашего пути к выживанию и поиску спасения.
Добро пожаловать в...

███████╗██╗  ██╗ ██████╗ ███╗   ███╗ █████╗ ██████╗  ██████╗  ██████╗ ███╗   ██╗███████╗██████╗
██╔════╝╚██╗██╔╝██╔═══██╗████╗ ████║██╔══██╗██╔══██╗██╔═══██╗██╔═══██╗████╗  ██║██╔════╝██╔══██╗
█████╗   ╚███╔╝ ██║   ██║██╔████╔██║███████║██████╔╝██║   ██║██║   ██║██╔██╗ ██║█████╗  ██║  ██║
█

Введите номер действия:  2



Вы начинаете исследовать капсулу.
Повреждены: радио, радар, изготовитель аптечек.
В попытках починить капсулу вы провели весь день, зачем?
у вас нет ресурсов для починки капсулы, попробуйте найти их вне её

ДЕНЬ 2
————————————————————————————————————————————————————
Статус капсулы: сломана.
Требуют починки: радио, радар(необходим для поиска редких ресурсов), изготовитель аптечек
Состояние сытости/жажды: 70% [███████───]
У вас на руках:
медь: 0
железо: 0
водоросли: 0
алмазы: 0
кораллы: 0
магнетит: 0
редкие ресурсы: 0
микросхема: 0
синтетические волокна: 0
нож: 0
острый раскалённый нож: 0
аптечки: 0

Вы можете совершить одно действие за день:
1. Исследовать окрестности капсулы
2. Попытаться починить капсулу
3. Создать предмет (не тратит день)


Введите номер действия:  3


Что создать?
1. Микросхема (2 коралла, 2 меди)
2. Кислородный баллон (2 железа)
3. Нож (2 железа)
4. Острый раскалённый нож (1 нож, 1 алмаз, 1 микросхема)
5. Аптечки (1 синтетическое волокно)
6. Переработать водоросли в волокна (2 водоросли)
7. Жареная рыба (1 водоросль) - восстановит 20% еды
8. Питательный коктейль (2 водоросли) - восстановит 30% еды
9. Консервы из морепродуктов (1 водоросль, требуется нож) - восстановит 25% еды
0. Вернуться назад.


...

Вы успешно срезаете несколько стеблей водорослей.
С помощью небольших скоплений водорослей вам удаётся создать импровизированную сеть. Благодаря ней вы ловите несколько рыб.
Вам удалось поймать рыбу и восстановить 25% запаса еды!
Стало темнеть. Пора возвращаться в капсулу.

ДЕНЬ 15
————————————————————————————————————————————————————
Статус капсулы: сломана.
Требуют починки: радио, радар(необходим для поиска редких ресурсов)
Состояние сытости/жажды: 83% [████████──]
У вас на руках:
медь: 4
железо: 4
водоросли: 4
алмазы: 2
кораллы: 4
магнетит: 2
редкие ресурсы: 0
микросхема: 1
синтетические волокна: 0
нож: 1
острый раскалённый нож: 0
аптечки: 0
Мутации, вызванные биосферой планеты приобрели необратимый характер, вас уже не спасти((

Вы можете совершить одно действие за день:
1. Исследовать окрестности капсулы
2. Попытаться починить капсулу
3. Создать предмет (не тратит день)


Введите номер действия:  3


Что создать?
1. Микросхема (2 коралла, 2 меди)
2. Кислородный баллон (2 железа)
3. Нож (2 железа)
4. Острый раскалённый нож (1 нож, 1 алмаз, 1 микросхема)
5. Аптечки (1 синтетическое волокно)
6. Переработать водоросли в волокна (2 водоросли)
7. Жареная рыба (1 водоросль) - восстановит 20% еды
8. Питательный коктейль (2 водоросли) - восстановит 30% еды
9. Консервы из морепродуктов (1 водоросль, требуется нож) - восстановит 25% еды
0. Вернуться назад.


KeyboardInterrupt: Interrupted by user

Введите номер (0 - для возврата):  0
